In [2]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Import the CRUD module
from crud import AnimalShelter

###########################
# Data Manipulation / Model
###########################
# Instantiate the CRUD class with correct credentials
shelter = AnimalShelter(username='aacuser', password='SNHU1234')

# Retrieve data from MongoDB and prepare DataFrame
try:
    data = shelter.read({})
    if data:
        df = pd.DataFrame.from_records(data)
        # Print DataFrame columns to debug
        print("DataFrame columns:", df.columns)
        
        # Check if '_id' column is in the DataFrame
        if '_id' in df.columns:
            df.drop(columns=['_id'], inplace=True)
        else:
            print("_id column not found in DataFrame.")
    else:
        print("No data retrieved from MongoDB.")
        df = pd.DataFrame()  # Create an empty DataFrame if no data
except Exception as e:
    print(f"Error during data retrieval or manipulation: {e}")
    df = pd.DataFrame()  # Create an empty DataFrame if there's an error

#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

# Path to your image file
image_filename = '/home/cheyenneherde_snhu/Desktop/Grazioso Salvare Logo.png' 
encoded_image = base64.b64encode(open(image_filename, 'rb').read()).decode()

app.layout = html.Div([
    # Add the logo image
    html.Img(src='data:image/png;base64,{}'.format(encoded_image), style={'width': '200px', 'height': 'auto'}),
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard'))),
    html.Hr(),

    # Add filtering options
    html.Div([
        html.Label('Filter by Rescue Type:'),
        dcc.Dropdown(
            id='rescue-type-dropdown',
            options=[
                {'label': 'Water Rescue', 'value': 'Water Rescue'},
                {'label': 'Mountain or Wilderness Rescue', 'value': 'Mountain or Wilderness Rescue'},
                {'label': 'Disaster or Individual Tracking', 'value': 'Disaster or Individual Tracking'}
            ],
            value=None,
            placeholder='Select a rescue type'
        ),
    ]),

    dash_table.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        row_selectable='single',
        selected_rows=[0],  # Default selection to first row
        page_size=10,
        style_table={'overflowX': 'auto'},
        style_data_conditional=[
            {
                'if': {'state': 'selected'},
                'backgroundColor': '#D2F3FF'
            }
        ]
    ),
    html.Br(),
    html.Hr(),
    html.Div(className='row',
             style={'display': 'flex'},
             children=[
                 html.Div(
                     id='graph-id',
                     className='col s12 m6',
                 ),
                 html.Div(
                     id='map-id',
                     className='col s12 m6',
                 )
             ])
])

#############################################
# Interaction Between Components / Controller
#############################################

# Callback to update data table based on the rescue type selected
@app.callback(
    Output('datatable-id', 'data'),
    [Input('rescue-type-dropdown', 'value')]
)
def update_table(rescue_type):
    if rescue_type:
        filtered_df = df[df['outcome_subtype'] == rescue_type]
    else:
        filtered_df = df
    return filtered_df.to_dict('records')

# Display the breeds of animal based on quantity represented in
# the data table
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")]
)
def update_graphs(viewData):
    dff = pd.DataFrame.from_dict(viewData)
    if not dff.empty:
        return [
            dcc.Graph(
                figure=px.pie(dff, names='breed', title='Distribution of Breeds for Selected Rescue Type')
            )
        ]
    else:
        return [html.P("No data available for the chart.")]

# This callback will update the geo-location chart for the selected data entry
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")]
)
def update_map(viewData, index):
    dff = pd.DataFrame.from_dict(viewData)
    
    if not dff.empty and len(dff.columns) > 14:
        row = index[0] if index else 0
        return [
            dl.Map(style={'width': '1000px', 'height': '500px'},
                   center=[30.75, -97.48], zoom=10, children=[
                dl.TileLayer(id="base-layer-id"),
                dl.Marker(position=[dff.iloc[row, 13], dff.iloc[row, 14]],
                          children=[
                              dl.Tooltip(dff.iloc[row, 4]),
                              dl.Popup([
                                  html.H1("Animal Name"),
                                  html.P(dff.iloc[row, 9])
                              ])
                          ])
            ])
        ]
    else:
        return [html.P("No data available for the map.")]

# Run the server
app.run_server(debug=True)

DataFrame columns: Index(['_id', 'rec_num', 'age_upon_outcome', 'animal_id', 'animal_type',
       'breed', 'color', 'date_of_birth', 'datetime', 'monthyear', 'name',
       'outcome_subtype', 'outcome_type', 'sex_upon_outcome', 'location_lat',
       'location_long', 'age_upon_outcome_in_weeks'],
      dtype='object')
Dash app running on http://127.0.0.1:13372/
